In [ ]:
from ipywidgets import Output
%run -m parse_data

# Road Query Demo
This notebook demonstrates querying the top 10 nearest road segments to a given coordinate using the Schema class.

In [11]:
%load_ext autoreload
%autoreload 2
from schema import Schema
from ipyleaflet import Map, basemaps, Polyline, Marker, Rectangle
from ipywidgets import Output

# Prompt for coordinates
lat,lon = map(float, "48.137394371914255, 11.575701088894027".split(", "))
radius = float(60)

schema = Schema()

out = Output()

center = (lat,lon)

mappy = Map(basemap = basemaps.OpenStreetMap.Mapnik, zoom = 16, center = center)

lines = []

def get_new_roads(event = None, location = None):
    with out:
        global lines
        out.clear_output()
        print(event)
        for line in lines:
            mappy.remove_layer(line)
        lines = []
        results, bbox = schema.nearest_road_distances(marker.location[1],marker.location[0], radius)

        lines+=[Rectangle(bounds = bbox, color = "#aa000033", fill = True)]

        if results:
            print('Top 10 nearest roads:')
            for i, (dist, row, _, _) in enumerate(results):
                line = Polyline(locations=[[row[7],row[6]],[row[9],row[8]]], color='red', fill=True)
                lines += [line]
            # dedupdict = {row[10]+row[11]: (dist, row, start, end) for dist, row, start, end in sorted(results, key = lambda x: x[0])}
            # results = sorted([dedupdict[row] for row in dedupdict.keys()], key=lambda x: x[0])[:10]
            for i, (dist, row, _, _) in enumerate(results, 1):
                print(f'{i}. {row[-2]} (maxspeed: {row[-1]}) - Distance: {dist:.2f} meters')
            lines += [Polyline(locations=results[0][2:4], color="blue", fill=False)]
            for line in lines:
                mappy.add_layer(line)
        else:
            print('No road found within radius.')

marker = Marker(location=center, draggable = True)

mappy.add_layer(marker)
marker.on_move(get_new_roads)
get_new_roads()

display(mappy, out)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Map(center=[48.137394371914255, 11.575701088894027], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Output()